In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 17314135
paper_name = 'loukin_saimi_2007' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [49]:
original_data = pd.read_excel('raw_data/06-7898SupplementalTable2.xlsx', sheet_name='Sheet1')

In [50]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4904 x 3


In [51]:
original_data.head()

,ORF,GENE,peak response
0,YJR073C,OPI3,813210
1,YJL127C,SPT10,263720
2,YAL026C,DRS2,157345
3,YBR035C,PDX3,130680
4,YJR139C,HOM6,113000


In [52]:
original_data['orf'] = original_data['ORF'].astype(str)

In [53]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [54]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [55]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, GENE, peak response, orf]
Index: []


In [56]:
original_data['data'] = 420 / pd.to_numeric(original_data['peak response'], errors='coerce')

In [57]:
original_data.set_index('orf', inplace=True)

In [58]:
original_data = original_data[['data']].copy()

In [60]:
original_data = original_data.groupby(original_data.index).mean()

In [61]:
original_data.shape

(4820, 1)

# Prepare the final dataset

In [63]:
data = original_data.copy()

In [64]:
dataset_ids = [1190]
datasets = datasets.reindex(index=dataset_ids)

In [65]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [66]:
data.head()

dataset_id,1190
data_type,value
orf,
YAL002W,0.141414
YAL004W,0.700000
YAL005C,0.636364
YAL007C,0.200957
YAL008W,0.777778


## Subset to the genes currently in SGD

In [67]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 26


In [68]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,1190
,data_type,value
gene_id,orf,
2,YAL002W,0.141414
1863,YAL004W,0.700000
4,YAL005C,0.636364
5,YAL007C,0.200957
6,YAL008W,0.777778


# Normalize

In [69]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [70]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [71]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id           1190          
data_type           value    valuez
gene_id orf                        
2       YAL002W  0.141414 -0.209984
1863    YAL004W  0.700000  0.026758
4       YAL005C  0.636364 -0.000212
5       YAL007C  0.200957 -0.184749
6       YAL008W  0.777778  0.059723

# Print out

In [72]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [73]:
from IO.save_data_to_db3 import *

In [74]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 17314135...
Inserting the new data...


100%|██████████| 1/1 [00:06<00:00,  6.91s/it]

Updating the data_modified_on field...
